In [ ]:
import argparse
import json
import logging
import sys
from os.path import join as pjoin

import h5py
import onnx
import torch
import torch.nn.functional as F

#from braindecode.models.deep4 import Deep4Net
from quantized_deep4 import QuantDeep4Net
from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.util import set_random_seeds

In [ ]:
datapath = "./processed_data/KU_mi_smt.h5"
dfile = h5py.File(datapath, 'r')
subj = 6
torch.cuda.set_device(0)
set_random_seeds(seed=20200205, cuda=True)

In [ ]:
def get_data(subj):
    dpath = '/s' + str(subj)
    X = dfile[pjoin(dpath, 'X')]
    Y = dfile[pjoin(dpath, 'Y')]
    return X[:], Y[:]

In [ ]:
# Get data for within-subject classification
X, Y = get_data(subj)
#print('Shape: ', X.shape)
X_train, Y_train = X[:200], Y[:200]
X_val, Y_val = X[200:300], Y[200:300]
X_test, Y_test = X[300:], Y[300:]

suffix = 's' + str(subj)
n_classes = 2
in_chans = X.shape[1]

# final_conv_length = auto ensures we only get a single output in the time dimension
model = QuantDeep4Net(in_chans=in_chans, n_classes=n_classes,
                 input_time_length=X.shape[2],
                 final_conv_length=1, split_first_layer=False).cuda()

# these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=1 * 0.01, weight_decay=0.5*0.001)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1, )

model.fit(X_train, Y_train, epochs=5, batch_size=16, scheduler='cosine', 
        validation_data=(X_val, Y_val))#, remember_best_column='valid_loss')

In [ ]:
test_loss = model.evaluate(X_test, Y_test)
print(test_loss)

In [ ]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
    
build_dir = "/workspace/finn"

In [ ]:
import brevitas.onnx as bo

bo.export_finn_onnx(model.network.cpu(), export_path=build_dir + "/auto_mai_subj6_export.onnx", input_t=torch.randn(1, 62, 1000, 1))

In [ ]:
showInNetron(build_dir + "/auto_mai_subj6_export.onnx")

In [ ]:
from finn.util.inference_cost import inference_cost
import json

cost_dict_path = build_dir + "/auto_mai_subj6_inference_cost.json"

inference_cost(build_dir + "/auto_mai_subj6_export.onnx", output_json=cost_dict_path, 
               output_onnx=build_dir + "/auto_mai_subj6_inference_cost.onnx",
               preprocess=True, discount_sparsity=True)

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = build_dir + "/auto_mai_subj6_export.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 10,
    synth_clk_period_ns = 10.0,
    board               = "ZCU104",
    #fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

In [ ]:
! ls {estimates_output_dir}

In [ ]:
! ls {estimates_output_dir}/report

In [ ]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

In [ ]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

In [ ]:
showInNetron(estimates_output_dir + '/intermediate_models/step_tidy_up.onnx')

In [ ]:
showInNetron(estimates_output_dir + '/intermediate_models/step_streamline.onnx')

In [ ]:
showInNetron(estimates_output_dir + '/intermediate_models/step_convert_to_hls.onnx')

In [ ]:
showInNetron(estimates_output_dir + '/intermediate_models/step_create_dataflow_partition.onnx')

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = build_dir + "/auto_mai_subj6_export.onnx"

rtlsim_output_dir = "output_ipstitch_ooc_rtlsim"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")

cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 100,
    synth_clk_period_ns = 10.0,
    board               = "ZCU104",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        '''Out of context synthesis fails for this model. Reason currently unknown.'''
        #build_cfg.DataflowOutputType.OOC_SYNTH,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

In [ ]:
! ls {rtlsim_output_dir}/stitched_ip

In [ ]:
! ls {rtlsim_output_dir}/report

In [ ]:
! cat {rtlsim_output_dir}/report/estimate_layer_resources_hls.json

In [ ]:
! cat {rtlsim_output_dir}/report/rtlsim_performance.json

In [ ]:
! cat {rtlsim_output_dir}/final_hw_config.json

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = build_dir + "/auto_mai_subj6_export.onnx"

final_output_dir = "output_6bit_10fps"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 50,
    synth_clk_period_ns = 10.0,
    board               = "ZCU104",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg)

In [ ]:
! ls {final_output_dir}/bitfile

In [ ]:
! ls {final_output_dir}/driver

In [ ]:
! ls {final_output_dir}/report

In [ ]:
! ls {final_output_dir}/deploy